In [1]:
from typing import Union, List
import numpy as np
import matplotlib.pyplot as plt
import string
import random
import re
import requests
import os
import textwrap

## True mapping

In [2]:
# create substitution cipher 

letters1 = list(string.ascii_lowercase)
letters2 = list(string.ascii_lowercase)

print(letters1)

# shuffle second list
random.shuffle(letters2)

true_mappings = {}
for k,v in zip(letters1, letters2):
    true_mappings[k] = v

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


This true mapping is the one only known, theoretically, by the sender and receiver, not by the intruder

In [3]:
print(true_mappings)

{'a': 'r', 'b': 'f', 'c': 'b', 'd': 'm', 'e': 'j', 'f': 'd', 'g': 'y', 'h': 'c', 'i': 'x', 'j': 't', 'k': 'e', 'l': 'i', 'm': 'h', 'n': 'a', 'o': 'g', 'p': 'q', 'q': 'z', 'r': 'l', 's': 'n', 't': 'u', 'u': 'v', 'v': 'w', 'w': 'k', 'x': 'p', 'y': 'o', 'z': 's'}


## Language Model

In [4]:
# leveraging ord function to get integers from a character to use as index
ord("a"), ord("b"), ord("c")

(97, 98, 99)

In [5]:
# markov matrix to store bigram probabilities
# we initialize with ones to consider "add-one smoothing"
M = np.ones((26,26))

# initial state distribution (unigrams probabilities)
pi = np.zeros(26)

def update_bigrams(ch1, ch2):
    i = ord(ch1) - 97
    j = ord(ch2) - 97
    M[i,j] += 1
    
def update_unigrams(ch):
    i = ord(ch) - 97
    pi[i] += 1
    
# get log-probability of a word/token
def get_word_prob(word):
    
    probs = []
    # first word index
    i = ord(word[0]) - 97
    probs.append(np.log(pi[i]))
    
    # rest of sentence
    for w_previous, w in zip(word, word[1:]):
        i = ord(w_previous) - 97
        j = ord(w) - 97
        probs.append(np.log(M[i,j]))
        
    # find log-probability
    return sum(probs)

# get log-probability of a document, which is a sequence of words
def get_sequence_prob(doc:Union[str, List]):
    
    if type(doc) == str:
        doc = doc.split()
        
    prob = 0
    for word in doc:
        prob += get_word_prob(word)
        
    return prob

## Creating a Language Model from Moby Dick Book

In [6]:
if not os.path.exists("moby_dick.txt"):
    print("downloading moby dick book...")
    r = requests.get("https://lazyprogrammer.me/course_files/moby_dick.txt")
    with open("moby_dick.txt", "w") as f:
        f.write(r.content.decode())

downloading moby dick book...


In [7]:
!ls

cipher.ipynb  moby_dick.txt


In [8]:
regex = re.compile('[^a-zA-Z]')

for line in open('moby_dick.txt', 'r'):
    line = line.rstrip()
    if line:
        # replace non-alpha characters with space
        line = regex.sub(' ', line) 
        tokens = line.lower().split()
        
        # update our language model 
        for token in tokens:
            # update first unigram letter
            ch0 = token[0]
            update_unigrams(ch0)
            
            # update bigrams for the other letters
            for ch1 in token[1:]:
                update_bigrams(ch0, ch1)
                ch0 = ch1

# normalize probabilities
pi /= pi.sum()
M /= M.sum(axis=1, keepdims=True)

In [9]:
M[:2], pi[:1]

(array([[7.04046861e-05, 2.76127179e-02, 3.36111972e-02, 4.38621195e-02,
         4.22428117e-04, 8.73018108e-03, 2.05018446e-02, 1.08704835e-02,
         4.75090822e-02, 2.95699682e-04, 1.45456082e-02, 1.10633924e-01,
         2.53034442e-02, 2.09721479e-01, 2.11214058e-04, 2.34306795e-02,
         7.04046861e-05, 1.08395055e-01, 9.77076234e-02, 1.42217466e-01,
         7.92756766e-03, 2.33602749e-02, 1.04198935e-02, 4.50589991e-04,
         2.94854826e-02, 2.63313526e-03],
        [6.00292826e-02, 2.66089503e-02, 6.36577758e-05, 5.72919982e-04,
         2.54949392e-01, 6.36577758e-05, 1.27315552e-04, 4.45604431e-04,
         3.79400344e-02, 5.47456872e-03, 6.36577758e-05, 1.20567827e-01,
         8.91208861e-04, 1.27315552e-04, 1.52333057e-01, 6.36577758e-05,
         6.36577758e-05, 5.88197848e-02, 1.89063594e-02, 7.25698644e-03,
         1.67483608e-01, 8.27551085e-04, 1.27315552e-04, 6.36577758e-05,
         8.60653129e-02, 6.36577758e-05]]), array([0.10945403]))

## Encoding Messages

In [10]:
### encode a message

# this is a random excerpt from Project Gutenberg's
# The Adventures of Sherlock Holmes, by Arthur Conan Doyle
# https://www.gutenberg.org/ebooks/1661

original_message = '''I then lounged down the street and found,
as I expected, that there was a mews in a lane which runs down
by one wall of the garden. I lent the ostlers a hand in rubbing
down their horses, and received in exchange twopence, a glass of
half-and-half, two fills of shag tobacco, and as much information
as I could desire about Miss Adler, to say nothing of half a dozen
other people in the neighbourhood in whom I was not in the least
interested, but whose biographies I was compelled to listen to.
'''

# Away they went, and I was just wondering whether I should not do well
# to follow them when up the lane came a neat little landau, the coachman
# with his coat only half-buttoned, and his tie under his ear, while all
# the tags of his harness were sticking out of the buckles. It hadn't
# pulled up before she shot out of the hall door and into it. I only
# caught a glimpse of her at the moment, but she was a lovely woman, with
# a face that a man might die for.

# My cabby drove fast. I don't think I ever drove faster, but the others
# were there before us. The cab and the landau with their steaming horses
# were in front of the door when I arrived. I paid the man and hurried
# into the church. There was not a soul there save the two whom I had
# followed and a surpliced clergyman, who seemed to be expostulating with
# them. They were all three standing in a knot in front of the altar. I
# lounged up the side aisle like any other idler who has dropped into a
# church. Suddenly, to my surprise, the three at the altar faced round to
# me, and Godfrey Norton came running as hard as he could towards me.

In [11]:
def encode_msg(msg):
    
    # lowercase everything and remove non-alpha charcaters
    msg = msg.lower()
    msg = regex.sub(" ", msg)
    
    coded_msg = []
    for ch in msg:
        coded_ch = ch
        if ch in true_mappings:
            coded_ch = true_mappings[ch]
        coded_msg.append(coded_ch)
            
    return "".join(coded_msg)

def decode_msg(msg, word_mapping):
    decoded_msg = []
    for ch in msg:
        decoded_ch = ch
        if ch in word_mapping:
            decoded_ch = word_mapping[ch]
        decoded_msg.append(decoded_ch)
            
    return "".join(decoded_msg)

In [12]:
encoded_msg = encode_msg(original_message)
print(encoded_msg)

x ucja igvayjm mgka ucj nuljju ram dgvam  rn x jpqjbujm  ucru ucjlj krn r hjkn xa r iraj kcxbc lvan mgka fo gaj krii gd ucj yrlmja  x ijau ucj gnuijln r cram xa lvffxay mgka ucjxl cglnjn  ram ljbjxwjm xa jpbcrayj ukgqjabj  r yirnn gd crid ram crid  ukg dxiin gd ncry ugfrbbg  ram rn hvbc xadglhruxga rn x bgvim mjnxlj rfgvu hxnn rmijl  ug nro agucxay gd crid r mgsja gucjl qjgqij xa ucj ajxycfgvlcggm xa kcgh x krn agu xa ucj ijrnu xaujljnujm  fvu kcgnj fxgylrqcxjn x krn bghqjiijm ug ixnuja ug  


## Genetic Evolutionary Algorithm to decode messages

In [13]:
def generate_dna_pool(n=20):
    dna_pool = []
    for _ in range(n):
        dna = list(string.ascii_lowercase)
        random.shuffle(dna)
        dna_pool.append(dna)
    
    return dna_pool

def procriate_offspring(dna_pool, n_children=3):
    
    offspring = []
    for parent in dna_pool:
        for _ in range(n_children):
            copy = parent.copy()
            i = np.random.randint(len(copy))
            j = np.random.randint(len(copy))
            
            # swap characters
            tmp = copy[i]
            copy[i] = copy[j]
            copy[j] = tmp
            
            offspring.append(copy)
            
    return offspring + dna_pool
        

In [32]:
def run_model(n_epochs=100):
    
    n_survivals = 5
    scores = np.zeros(n_epochs)
    best_dna = None
    best_map = None
    best_score = float('-inf')
    
    dna_pool = generate_dna_pool()
    
    for i in range(n_epochs):
        if i > 0:
            dna_pool = procriate_offspring(dna_pool, n_survivals)

        # calculate score for each dna
        dna2score = {}
        for dna in dna_pool:
            # build a map from current dna sequence
            current_map = {}
            for k,v in zip(letters1, dna):
                current_map[k] = v

            # decode using current map    
            decoded_msg = decode_msg(encoded_msg, current_map)
            score = get_sequence_prob(decoded_msg)

            # store this result
            dna2score[''.join(dna)] = score

            # check if this score is better than the best
            if score > best_score:
                best_score = score
                best_dna = dna
                best_map = current_map

        scores[i] = np.mean(list(dna2score.values()))

        # keep the best DNAs, survival of the fittest, using n_survivals
        sorted_dna = sorted(dna2score.items(), key=lambda x: x[1], reverse=True)
        dna_pool = [list(k) for k,v in sorted_dna[:n_survivals]]
#         [list(k) for k, v in sorted_dna[:5]]

        if i % 200 == 0:
            print("iter:", i, "score:", scores[i], "best so far:", best_score)
        
    return best_dna, best_map, best_score, scores

In [33]:
# Run the evolution!
n_epochs = 1000
best_dna, best_map, best_score, scores = run_model(n_epochs)

iter: 0 score: -2062.184439056681 best so far: -1774.5103379069421
iter: 200 score: -1036.0586442040906 best so far: -929.5902922650557
iter: 400 score: -1021.5983857358721 best so far: -929.5902922650557
iter: 600 score: -1066.9281144808915 best so far: -929.5902922650557
iter: 800 score: -1029.7864992265195 best so far: -929.5902922650557


In [34]:
# use best score
decoded_msg = decode_msg(encoded_msg, best_map)

print("LL of decoded with best mapping: ", get_sequence_prob(decoded_msg))
print("LL of original mapping: ", get_sequence_prob(regex.sub(" ", original_message.lower())))

for true, v in true_mappings.items():
    pred = best_map[v] # best map is a reverse map
    if true != pred:
        print(f"true: {true}, pred: {pred}")

LL of decoded with best mapping:  -929.5902922650557
LL of original mapping:  -933.0312453751817
true: k, pred: q
true: q, pred: z
true: z, pred: k


In [38]:
print("decoded message:\n\n", textwrap.fill(decoded_msg)) 

print("\noriginal message:\n\n", original_message)

decoded message:

 i then lounged down the street and found  as i expected  that there
was a mews in a lane which runs down by one wall of the garden  i lent
the ostlers a hand in rubbing down their horses  and received in
exchange twopence  a glass of half and half  two fills of shag tobacco
and as much information as i could desire about miss adler  to say
nothing of half a doken other people in the neighbourhood in whom i
was not in the least interested  but whose biographies i was compelled
to listen to

original message:

 I then lounged down the street and found,
as I expected, that there was a mews in a lane which runs down
by one wall of the garden. I lent the ostlers a hand in rubbing
down their horses, and received in exchange twopence, a glass of
half-and-half, two fills of shag tobacco, and as much information
as I could desire about Miss Adler, to say nothing of half a dozen
other people in the neighbourhood in whom I was not in the least
interested, but whose biographies 